In [95]:
import GetPropertiesAPI as GP
from collections import Counter
import pandas as pd
import importlib
importlib.reload(GP)

<module 'GetPropertiesAPI' from '/Users/sreejithmenon/Google Drive/PythonCode/AnimalPhotoBias/script/GetPropertiesAPI.py'>

In [84]:
aid_list = []
gidAidDict = {}
for gid in range(1,500):
    aid = GP.getAnnotID(gid)
    aid_list.append(aid)
    gidAidDict[gid] = aid
    
aid_list = list(filter(lambda x: x != None,aid_list))

In [85]:
aids = []
for aid_l in aid_list:
    for aid in aid_l:
        aids.append(aid)

In [99]:
aidContribTupList = []
for aid in aids:
    contrib = GP.getImageFeature(aid,'image_contributor_tag')
    aidContribTupList.append((aid,contrib[0]))

In [86]:
aidNidMap = {}
aidNamesMap = {}
aidNidTupList = [] # modified

for aid in aids:
    nid = GP.getImageFeature(aid,'nids')
    aidNidMap[aid] = nid
    aidNidTupList.append((aid,nid[0]))

In [87]:
nids = []
for aid in aidNidMap.keys():
    nids.append(aidNidMap[aid][0])

In [88]:
nids = list(filter(lambda x : x > 0,nids))

In [89]:
counter_nid,counter_name = Counter(nids),Counter(names)

In [90]:
gidAidDict
aidGidTupList = [] # key : aid and value : gid # modified
for gid in gidAidDict.keys():
    if gidAidDict[gid] != None:
        for aid in gidAidDict[gid]:
            aidGidTupList.append((aid,gid))

In [91]:
aidGidDf = pd.DataFrame(aidGidTupList,columns = ['AID','GID']) 
aidNidDf = pd.DataFrame(aidNidTupList,columns = ['AID','NID']) 
aidContribDf = pd.DataFrame(aidContribTupList,columns = ['AID','CONTRIBUTOR'])
aidNidDf = aidNidDf[(aidNidDf['NID']>0)]

In [92]:
aidGidNidDf = pd.merge(aidGidDf,aidNidDf,left_on = 'AID',right_on = 'AID')
aidGidNidContribDf = pd.merge(aidGidNidDf,aidContribDf,left_on = 'AID',right_on = 'AID')

In [107]:
aidGidNidContribDf.to_csv('results.csv',index=False)